## CUCM Certificate Cleanup

### Process Overview
1. Collect all the trust certificates from CUCM using admin CLI
2. Inspect each certificate for validity
3. Delete the expired certificates using the admin CLI
4. Restart the tomcat service

In [1]:
import time
import sys
import threading
from ntpath import basename
import getpass
from multiprocessing.pool import ThreadPool
import itertools
import paramiko
from paramiko_expect import SSHClientInteraction
import os.path
import re
import pandas as pd
pd.set_option('display.max_colwidth', None)

from datetime import timezone
from email.utils import parsedate_to_datetime
from email.utils import localtime

### 1. Collect all the trust certificates from CUCM using admin CLI

In [26]:
#Create a SSH connection to the CUCM
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect('10.201.115.10', username='Administrator', password='C!sc0123')
interact = SSHClientInteraction(ssh, timeout=90, display=True)
interact.expect('admin:')

Command Line Interface is starting up, please wait ...

   Welcome to the Platform Command Line Interface

VMware Installation:
	2 vCPU: Intel(R) Xeon(R) Gold 6142 CPU @ 2.60GHz
	Disk 1: 120GB, Partitions aligned
	4096 Mbytes RAM

admin:

0

In [27]:
#Store the output of the SSH commands
output = []
output.clear()

#Display list of trust certificates
interact.send("show cert list trust")
interact.expect('admin:')
output.append(interact.current_output)
certslist = interact.current_output

show cert list trust

tomcat-trust/astrng-RCDN-WIN12-SRV1-CA.pem: Trust Certificate
tomcat-trust/rcdn-astp-cucm-c2-1-EC.astrng.cisco.com.pem: Trust Certificate
tomcat-trust/rcdn-astp-cucm-c2-1.astrng.cisco.com.pem: Trust Certificate
tomcat-trust/VeriSign_Class_3_Secure_Server_CA_-_G3.pem: Trust Certificate
ipsec-trust/rcdn-astp-cucm-c2-1.astrng.cisco.com.pem: Trust Certificate
CallManager-trust/Cisco_Manufacturing_CA_SHA2.pem: 
CallManager-trust/Cisco_Root_CA_2048.pem: 
CallManager-trust/CAPF-4d8b6210.pem: 
CallManager-trust/astrng-RCDN-WIN12-SRV1-CA.pem: Internal CA Root cert
CallManager-trust/CAPF-8504d067.pem: 
CallManager-trust/Cisco_Root_CA_M2.pem: 
CallManager-trust/CAPF-d15cd4d2.pem: 
CallManager-trust/CAP-RTP-001.pem: 
CallManager-trust/CAP-RTP-002.pem: 
CallManager-trust/cms0.astrng.cisco.com.pem: Signed Certificate
CallManager-trust/ACT2_SUDI_CA.pem: 
CallManager-trust/Cisco_Manufacturing_CA.pem: 
CAPF-trust/CAPF-855e024d.pem: 
CAPF-trust/Cisco_Manufacturing_CA_SHA2.pem: 
CAP

### 2. Inspect each certificate for validity

In [28]:
#Extract the certifcate names using regex from the stored output
certnames = re.findall(r'\n(.*?)\:', certslist)
certnames.pop()

#Display certs before and after modification (esp. nonalphanumeric characters before '-')

df_cert_comp = pd.DataFrame(columns = ['Cert_Before','Cert_After'])
certcnt = 0
for i in range(len(certnames)):
    orig = certnames[i]
    repl = re.sub(r'[^a-zA-Z\d\s:]\-[^a-zA-Z\d\s:]','_', certnames[i])
    #certnames[i] = re.sub(r'[^a-zA-Z\d\s:]\-[^a-zA-Z\d\s:]','_', certnames[i])
    #certissue = re.sub(r'[^a-zA-Z\d\s:]\-[^a-zA-Z\d\s:]',' ', certissue)
    #print(certnames[i])
    df_cert_temp = pd.DataFrame({'Cert_Before' : [orig], 'Cert_After' : [repl]})
    df_cert_comp = df_cert_comp.append(df_cert_temp, ignore_index=True)
    certcnt+=1
print('Number of trust certificate', certcnt)
df_cert_comp

Number of trust certificate 32


,Cert_Before,Cert_After
0,tomcat-trust/astrng-RCDN-WIN12-SRV1-CA.pem,tomcat-trust/astrng-RCDN-WIN12-SRV1-CA.pem
1,tomcat-trust/rcdn-astp-cucm-c2-1-EC.astrng.cisco.com.pem,tomcat-trust/rcdn-astp-cucm-c2-1-EC.astrng.cisco.com.pem
2,tomcat-trust/rcdn-astp-cucm-c2-1.astrng.cisco.com.pem,tomcat-trust/rcdn-astp-cucm-c2-1.astrng.cisco.com.pem
3,tomcat-trust/VeriSign_Class_3_Secure_Server_CA_-_G3.pem,tomcat-trust/VeriSign_Class_3_Secure_Server_CA_G3.pem
4,ipsec-trust/rcdn-astp-cucm-c2-1.astrng.cisco.com.pem,ipsec-trust/rcdn-astp-cucm-c2-1.astrng.cisco.com.pem
5,CallManager-trust/Cisco_Manufacturing_CA_SHA2.pem,CallManager-trust/Cisco_Manufacturing_CA_SHA2.pem
6,CallManager-trust/Cisco_Root_CA_2048.pem,CallManager-trust/Cisco_Root_CA_2048.pem
7,CallManager-trust/CAPF-4d8b6210.pem,CallManager-trust/CAPF-4d8b6210.pem
8,CallManager-trust/astrng-RCDN-WIN12-SRV1-CA.pem,CallManager-trust/astrng-RCDN-WIN12-SRV1-CA.pem
9,CallManager-trust/CAPF-8504d067.pem,CallManager-trust/CAPF-8504d067.pem


In [29]:
#Dataframe to create new columns matching the cert names
df_cert = pd.DataFrame(columns = ['Cert_Name', 'Valid_From', 'Valid_To', 'Validity'])

#Capture local time in UTC
lt = localtime()
ltimestamp = lt.replace(tzinfo=timezone.utc).timestamp()

#For each certificate, extract valid from, to and validate the certificate expiry
count = 0 
for i in range(len(certnames)):
    output.clear()
    #
    command = 'show cert trust ' + certnames[i]
    interact.send(command)
    interact.expect("Press <enter> for 1 line, <space> for one page, or <q> to quit",timeout = 5)
    interact.send("q")
    output.append(interact.current_output)
    interact.expect('admin:',timeout = 5)
    #interact.expect('Press <enter> for 1 line, <space> for one page, or <q> to quit')
    #output.append(interact.current_output)
    #interact.expect('admin:')
    #interact.send(" ")
    #output.append(interact.current_output)
    
    #Extract Validity From and To from the Certificate
    output_str = ' '.join(str(elem) for elem in output)
    valid_from = re.findall(r'From: (.*?)\n', output_str)
    valid_from_str = ' '.join(str(elem) for elem in valid_from)
    valid_to = re.findall(r'To:   (.*?)\n', output_str)
    valid_to_str = ' '.join(str(elem) for elem in valid_to)
    
    print(certnames[i],valid_from_str,valid_to_str)
    
    #Capture the Cert Name, Valid From and Valid To into a dataframe and insert the values
    df_cert_ind = pd.DataFrame({'Cert_Name' : [certnames[i]], 'Valid_From' : [valid_from_str], 'Valid_To' : [valid_to_str]})
    df_cert = df_cert.append(df_cert_ind, ignore_index=True)
    
    #Validate the timestamp for Valid To field
    if valid_to_str != "":
        dt = parsedate_to_datetime(valid_to_str)
        timestamp = dt.replace(tzinfo=timezone.utc).timestamp()
    else:
        valid_to_str = ltimestamp

    #Logic to validate the certificate expiry
    if timestamp > ltimestamp:
        df_cert.Validity[i] = "Valid"
    else:
        df_cert.Validity[i] = "Expired"
    
    count = count + 1

    print (df_cert)

show cert trust tomcat-trust/astrng-RCDN-WIN12-SRV1-CA.pem
[
  Version: V3
  Serial Number: 3ADA9D864E5E51AE459F17C9CB619D3A
  SignatureAlgorithm: SHA256withRSA (1.2.840.113549.1.1.11)
  Issuer Name: CN=astrng-RCDN-WIN12-SRV1-CA, DC=astrng, DC=cisco, DC=com
  Validity From: Thu Sep 08 16:22:32 CDT 2016
           To:   Tue Sep 08 16:32:32 CDT 2026
  Subject Name: CN=astrng-RCDN-WIN12-SRV1-CA, DC=astrng, DC=cisco, DC=com
  Key: RSA (1.2.840.113549.1.1.1)
    Key value: 3082010a0282010100b49d11cde267ee4ba69d8c0bc8d679d00dd24a9eb1b164a6ea976be443535440e18549f6d7c510b73473e401b1cac7b95185e53f2a28ee3f6c8335796e5a6d4674dda1085d55633379fc2a7654a202b7ac392708e55b4a0280507eedd2b71545456718be422cc1765920fdbe979137b6392e8dec9c19f28de30136fd5c6887c8ee4c9103f9f4d1266d291baf6349dec2554299b9e2f4279c67f4ffc86f05b0af49255f7a6606f8789829a3f543e47afd1391189a5aab550a45653f0067ef7843998b8ff07a2acdc6ecf4692e9f277fa1fabd1348969c45e756c5b8d2a18b4b45cae0d3ccd9e84ab6c7032644f767dd4a0bfbf8ca37069f3aa199fb40032b4

show cert trust CallManager-trust/Cisco_Manufacturing_CA_SHA2.pem
[
  Version: V3
  Serial Number: 2
  SignatureAlgorithm: SHA256withRSA (1.2.840.113549.1.1.11)
  Issuer Name: CN=Cisco Root CA M2, O=Cisco
  Validity From: Mon Nov 12 07:50:58 CST 2012
           To:   Thu Nov 12 07:00:17 CST 2037
  Subject Name: CN=Cisco Manufacturing CA SHA2, O=Cisco
  Key: RSA (1.2.840.113549.1.1.1)
    Key value: 3082010a0282010100f4364b42023267de493df2153bc14569e9094e16b948b4471ee82a5b7d8a5e2dd51e65db80a3e4b4a2dcd3949c12d8194c859b5a72ef6f5297e65bdc3b9d0a3dcd54f77b23ee845ffaffd4c4755067ef9cdbe427d5f84ea577e7c79ee3e217206ef870c3251777ef73a9fbde7b21da16c8fbc3f211d1d47246149dc924a060d1449c2f8242c257aef7c5eaff23e502a0611316bb6b6fdfa92999034bd9206d5b05f59963c66d49e4f12a0dde5b29d5fb112569b1ea4c331859ffb6a1bb38601e6520d4db6201f24444cfe93f17afa4ed0f4877eb9e0e507716fb599e06efe372d96e30aa697928d5b6ba1fe6fb7ea5b9028348900008ec2f4a9ccd3df268d5ef0203010001
  Extensions: 7 present
  [ 
     Extension: KeyUsage (OI

     Extension: KeyUsage (OID.2.5.29.15)
     Critical: false
     Usages: digitalSignature, keyCertSign, cRLSign, 
  ]
  [ 
Press <enter> for 1 line, <space> for one page, or <q> to quitadmin:
admin:CallManager-trust/astrng-RCDN-WIN12-SRV1-CA.pem Thu Sep 08 16:22:32 CDT 2016 Tue Sep 08 16:32:32 CDT 2026
                                                  Cert_Name  \
0                tomcat-trust/astrng-RCDN-WIN12-SRV1-CA.pem   
1  tomcat-trust/rcdn-astp-cucm-c2-1-EC.astrng.cisco.com.pem   
2     tomcat-trust/rcdn-astp-cucm-c2-1.astrng.cisco.com.pem   
3   tomcat-trust/VeriSign_Class_3_Secure_Server_CA_-_G3.pem   
4      ipsec-trust/rcdn-astp-cucm-c2-1.astrng.cisco.com.pem   
5         CallManager-trust/Cisco_Manufacturing_CA_SHA2.pem   
6                  CallManager-trust/Cisco_Root_CA_2048.pem   
7                       CallManager-trust/CAPF-4d8b6210.pem   
8           CallManager-trust/astrng-RCDN-WIN12-SRV1-CA.pem   

                     Valid_From                      Valid_To V

Press <enter> for 1 line, <space> for one page, or <q> to quitadmin:
admin:CallManager-trust/CAPF-d15cd4d2.pem Fri Aug 30 16:32:08 CDT 2013 Wed Aug 29 16:32:07 CDT 2018
                                                   Cert_Name  \
0                 tomcat-trust/astrng-RCDN-WIN12-SRV1-CA.pem   
1   tomcat-trust/rcdn-astp-cucm-c2-1-EC.astrng.cisco.com.pem   
2      tomcat-trust/rcdn-astp-cucm-c2-1.astrng.cisco.com.pem   
3    tomcat-trust/VeriSign_Class_3_Secure_Server_CA_-_G3.pem   
4       ipsec-trust/rcdn-astp-cucm-c2-1.astrng.cisco.com.pem   
5          CallManager-trust/Cisco_Manufacturing_CA_SHA2.pem   
6                   CallManager-trust/Cisco_Root_CA_2048.pem   
7                        CallManager-trust/CAPF-4d8b6210.pem   
8            CallManager-trust/astrng-RCDN-WIN12-SRV1-CA.pem   
9                        CallManager-trust/CAPF-8504d067.pem   
10                    CallManager-trust/Cisco_Root_CA_M2.pem   
11                       CallManager-trust/CAPF-d15cd4d2.pem   

show cert trust CallManager-trust/cms0.astrng.cisco.com.pem
[
  Version: V3
  Serial Number: 19000000174ADA04D849D82AA4000000000017
  SignatureAlgorithm: SHA256withRSA (1.2.840.113549.1.1.11)
  Issuer Name: CN=astrng-RCDN-WIN12-SRV1-CA, DC=astrng, DC=cisco, DC=com
  Validity From: Fri Sep 08 15:28:30 CDT 2017
           To:   Sun Sep 08 15:38:30 CDT 2019
  Subject Name: CN=cms0.astrng.cisco.com
  Key: RSA (1.2.840.113549.1.1.1)
    Key value: 3082010a0282010100e4856172bea83082a71927859e828e63af301027cf2d5c6b81b0cdd90e9a8c4457487c4c25b876635dee4d29a287bfbebe7ae10e6cec5b5e50d01bd3bb28ddff9679e450c897e81db5f851234d1ff54f46d352ee6b0c9e840836fbc3632a15c8fb54074c5d1304f56bd8c0fb6b0f242f53c669b7e16b17d908fa1daa249e7fa330304d0434e9f63164c6efece1233bc5b2d132ccfeac367767db6ae41069317af38b3744c40531d2a735c01cbca3bd0e46d4fa17f617d6a55d59a37d0aba332b56dbc4be762398ca132ed0e1689f79e348998448c0ee2a0889652f5a55f2e8a6c628fd2d61589e8d1c8b3c6b080b6d1c388959ced7ca1189294b9cd9cae491cb0203010001
  Extensions

...

show cert trust CAPF-trust/Cisco_Root_CA_M2.pem
[
  Version: V3
  Serial Number: 1
  SignatureAlgorithm: SHA256withRSA (1.2.840.113549.1.1.11)
  Issuer Name: CN=Cisco Root CA M2, O=Cisco
  Validity From: Mon Nov 12 07:00:18 CST 2012
           To:   Thu Nov 12 07:00:18 CST 2037
  Subject Name: CN=Cisco Root CA M2, O=Cisco
  Key: RSA (1.2.840.113549.1.1.1)
    Key value: 3082010a0282010100d8062521134f8d3a757ce93dc874413b4da065f9c2bf9484ef6b0b7187d458c260322cbaccbabb6334dbb312426a798586b3e87fc9928be3a73745d39483686520db8ad44341dfca4cd4d710cd682795724e744a54f5da71e9a5af4d0cb16c31fb0cde73829a50095e0ee335beba4ec2cae06ec8842d8ab3ee92ec0482e4c76e4d18b5e964cb6486d2f0b8e159163aa6264805ee63291e1583ef1e79c1820029d4568f0a6dc2c858392d63b871803ae1dcebb31352220551470088876dedd13cd7b8da857924c62a5d1df2834d811fa7cdd4198eab6940e6fb070af28e653e6de53dc132ee06262decbb1eb14562512bf0591731721acc0074c30cbb84ca73ad0203010001
  Extensions: 3 present
  [ 
     Extension: KeyUsage (OID.2.5.29.15)
     Critical: t

show cert trust CAPF-trust/CAPF-d15cd4d2.pem
[
  Version: V3
  Serial Number: 506F859CD4F1739DB396BD396F8E043B
  SignatureAlgorithm: SHA1withRSA (1.2.840.113549.1.1.5)
  Issuer Name: L=Richardon, ST=TX, CN=CAPF-d15cd4d2, OU=CA, O=AS Video Team, C=US
  Validity From: Fri Aug 30 16:32:08 CDT 2013
           To:   Wed Aug 29 16:32:07 CDT 2018
  Subject Name: L=Richardon, ST=TX, CN=CAPF-d15cd4d2, OU=CA, O=AS Video Team, C=US
  Key: RSA (1.2.840.113549.1.1.1)
    Key value: 30818902818100b3246b1fdcb332c0ee0ee9e8d92a42d7c7fa19485bab15503d9e923752f2a5205630b73f64e21f42d96b47b9f9f33dda2e38f0947e4e46e32333fbff0c9f9002217fb5342920408f893600a48f8ee80e549555ebb080554c0c0e178ee3274500452b49bd04490df0786c8dec9231f1712340ed21a8fde5c5a5eae62ff53483d50203010001
  Extensions: 3 present
  [ 
     Extension: KeyUsage (OID.2.5.29.15)
     Critical: false
     Usages: digitalSignature, keyEncipherment, keyCertSign, 
  ]
  [ 
     Extension: ExtKeyUsageSyntax (OID.2.5.29.37)
Press <enter> for 1 line, <space>

show cert trust CAPF-trust/CAP-RTP-001.pem
[
  Version: V3
  Serial Number: 7612F960153D6F9F4E42202032B72356
  SignatureAlgorithm: SHA1withRSA (1.2.840.113549.1.1.5)
  Issuer Name: CN=CAP-RTP-001, O=Cisco Systems
  Validity From: Thu Feb 06 17:27:13 CST 2003
           To:   Mon Feb 06 17:36:34 CST 2023
  Subject Name: CN=CAP-RTP-001, O=Cisco Systems
  Key: RSA (1.2.840.113549.1.1.1)
    Key value: 308201080282010100ac55bbed18de9b8709ffbc8f2d509ab83a21c1967fdea7f4b0969694b780cc196a463da51654a28f475d903b5f104a3d54a981389b2fc7ac49956262b81c1430385345bb2e273fa7a646860573ce5c998d55de78aa5a5cfe14037d695bac816409c6211f0b3bbf09cfb0bbb2d4ac362f670fd145f1620852b31f07e2f1aa74f150367632eda289e374af0c5b78ce7dfb9fc8ebbe546ecf4c7799d6dc0447476c0f36e58a3b6bcb24d76b6c84c27f61d326be7cb4a660cd65799e1e3a848153b7505527e865423be2b5cb5754535aa9609358b6a2e4aa3ef081c7068ec1dd1ebdda53e6f0d6e2e0486b109f131678c696a3cfba84cc7094034fc1eb9f81931acb020103
  Extensions: 5 present
  [ 
     Extension: KeyUsage (OID.2.

show cert trust CAPF-trust/ACT2_SUDI_CA.pem
[
  Version: V3
  Serial Number: 61096E7D00000000000C
  SignatureAlgorithm: SHA1withRSA (1.2.840.113549.1.1.5)
  Issuer Name: CN=Cisco Root CA 2048, O=Cisco Systems
  Validity From: Thu Jun 30 12:56:57 CDT 2011
           To:   Mon May 14 15:25:42 CDT 2029
  Subject Name: CN=ACT2 SUDI CA, O=Cisco
  Key: RSA (1.2.840.113549.1.1.1)
    Key value: 3082010a0282010100d26e65dd31c8c40f6d37f852e6a47fe94651a5d77ef5a13625b16436386dea07c729de3bec0912e5702d52ce68c43615b7fcc46ec959ab7f8b47aaaba8a29056ee8962f1f4e5460b06a0a3dfcb3be8d2e4e77f74e56ce9f6945c99108fb97f9aead1c5fea46e3a2278e2675e0d8668dea3c2a71ce9ad62433dc94ce2260a1ee98c40caa682683ec60847228a164bcd0139c762ca711b78fe6b67ba91b9ec0a3299754e223759d04d8d98e7a5f8e0eba17e3f849a0e419b0577468f5de855e17e1321562eb1638fdedf2f6fbca1ab95e3742bbe77f777f19fa978fb7b0fb3fec5e55ed10e8d25c9511b323048f9dd175df6d2701e4cbc9de6a9b334beafe91d550203010001
  Extensions: 7 present
  [ 
     Extension: KeyUsage (OID.2.5.29.15

show cert trust CAPF-trust/Cisco_Manufacturing_CA.pem
[
  Version: V3
  Serial Number: 6A6967B3000000000003
  SignatureAlgorithm: SHA1withRSA (1.2.840.113549.1.1.5)
  Issuer Name: CN=Cisco Root CA 2048, O=Cisco Systems
  Validity From: Fri Jun 10 17:16:01 CDT 2005
           To:   Mon May 14 15:25:42 CDT 2029
  Subject Name: CN=Cisco Manufacturing CA, O=Cisco Systems
  Key: RSA (1.2.840.113549.1.1.1)
    Key value: 308201080282010100a0c5f7dc96943515f1f4994ebb9b41e17ddb791691bbf354f2414a94326262c923f79ae7bb9b79e807294e30f5ae1bc5215646b0f8f4e68e81b816cca89b85d24281db7ccb94a91161121c5cea33201c9a16a77ddb99066ae236afecf80aff986707f430eea5f8881aaae8c73c1cceee48fdcd5c37f186939e3d71757d34ee4b14a9c0297b0510ef879e693130f548363fd8abce15e2e8589f3e6271048726a415620125aad5dfc9c95bb8c9a1077bbe6892939320a86cbd1575d3445d454beca8da60c7d8c8d5c8ed41e1f55f578e53329349d5d90ff836aa07c43241c5a7af1d19fff67399395a73676213340d1f5e957052641706ec535c5cdb6aea350041020103
  Extensions: 10 present
  [ 
     Extension

In [30]:
#Display all certificates with valid duration
df_cert

,Cert_Name,Valid_From,Valid_To,Validity
0,tomcat-trust/astrng-RCDN-WIN12-SRV1-CA.pem,Thu Sep 08 16:22:32 CDT 2016,Tue Sep 08 16:32:32 CDT 2026,Valid
1,tomcat-trust/rcdn-astp-cucm-c2-1-EC.astrng.cisco.com.pem,Thu Aug 17 00:13:49 CDT 2017,Tue Aug 16 00:13:48 CDT 2022,Valid
2,tomcat-trust/rcdn-astp-cucm-c2-1.astrng.cisco.com.pem,Thu Aug 17 00:13:41 CDT 2017,Tue Aug 16 00:13:40 CDT 2022,Valid
3,tomcat-trust/VeriSign_Class_3_Secure_Server_CA_-_G3.pem,,,Valid
4,ipsec-trust/rcdn-astp-cucm-c2-1.astrng.cisco.com.pem,Thu Aug 17 00:13:55 CDT 2017,Tue Aug 16 00:13:54 CDT 2022,Valid
5,CallManager-trust/Cisco_Manufacturing_CA_SHA2.pem,Mon Nov 12 07:50:58 CST 2012,Thu Nov 12 07:00:17 CST 2037,Valid
6,CallManager-trust/Cisco_Root_CA_2048.pem,Fri May 14 15:17:12 CDT 2004,Mon May 14 15:25:42 CDT 2029,Valid
7,CallManager-trust/CAPF-4d8b6210.pem,Tue Nov 08 21:15:01 CST 2016,Sun Nov 07 21:15:00 CST 2021,Expired
8,CallManager-trust/astrng-RCDN-WIN12-SRV1-CA.pem,Thu Sep 08 16:22:32 CDT 2016,Tue Sep 08 16:32:32 CDT 2026,Valid
9,CallManager-trust/CAPF-8504d067.pem,Thu Aug 17 00:14:09 CDT 2017,Tue Aug 16 00:14:08 CDT 2022,Valid


In [31]:
#Subset and Display expired certificates
df_cert_exp = df_cert[df_cert.Validity == "Expired"]
df_cert_exp

,Cert_Name,Valid_From,Valid_To,Validity
7,CallManager-trust/CAPF-4d8b6210.pem,Tue Nov 08 21:15:01 CST 2016,Sun Nov 07 21:15:00 CST 2021,Expired
11,CallManager-trust/CAPF-d15cd4d2.pem,Fri Aug 30 16:32:08 CDT 2013,Wed Aug 29 16:32:07 CDT 2018,Expired
14,CallManager-trust/cms0.astrng.cisco.com.pem,Fri Sep 08 15:28:30 CDT 2017,Sun Sep 08 15:38:30 CDT 2019,Expired
17,CAPF-trust/CAPF-855e024d.pem,Tue Nov 08 22:55:45 CST 2016,Wed Nov 08 23:05:45 CST 2017,Expired
20,CAPF-trust/CAPF-4d8b6210.pem,Tue Nov 08 21:15:01 CST 2016,Sun Nov 07 21:15:00 CST 2021,Expired
22,CAPF-trust/CAPF-767e97c9.pem,Wed Nov 09 11:11:31 CST 2016,Mon Nov 08 11:11:30 CST 2021,Expired
24,CAPF-trust/CAPF-481bb1c7.pem,Tue Nov 08 22:39:08 CST 2016,Sun Nov 07 22:39:07 CST 2021,Expired
25,CAPF-trust/CAPF-d15cd4d2.pem,Fri Aug 30 16:32:08 CDT 2013,Wed Aug 29 16:32:07 CDT 2018,Expired
26,CAPF-trust/CAPF-f532026c.pem,Wed Dec 07 14:22:37 CST 2016,Mon Dec 06 14:22:36 CST 2021,Expired
30,CAPF-trust/CAPF-2f592ecd.pem,Wed Oct 12 13:29:16 CDT 2016,Thu Oct 12 13:39:16 CDT 2017,Expired


### 3. Delete the expired certificates using the admin CLI

In [ ]:
# Delete the expired certificates
delcount = 0
for cert in df_cert_exp["Cert_Name"]:
    unit = cert.rpartition('/')[0]
    name = cert.rpartition('/')[2]
    if unit == "CallManager-trust":
        output.clear()
        interact.send(" ")
        interact.expect('admin:')
        command = "set cert delete CallManager" + name
        interact.send(command)
        time.sleep(5)
        interact.send("yes")
        interact.send(" ")
        interact.expect('admin:', timeout = 5)
    else if unit == "CAPF-trust":
        output.clear()
        interact.send(" ")
        interact.expect('admin:')
        command = "set cert delete CAPF" + name
        interact.send(command)
        time.sleep(5)
        interact.send("yes")
        interact.send(" ")
        interact.expect('admin:', timeout = 5)
    else if unit == "tomcat-trust":
        output.clear()
        interact.send(" ")
        interact.expect('admin:')
        command = "set cert delete tomcat" + name
        interact.send(command)
        time.sleep(5)
        interact.send("yes")
        interact.send(" ")
        interact.expect('admin:', timeout = 5)
    else if unit == "ipsec-trust":
        output.clear()
        interact.send(" ")
        interact.expect('admin:')
        command = "set cert delete ipsec" + name
        interact.send(command)
        time.sleep(5)
        interact.send("yes")
        interact.send(" ")
        interact.expect('admin:', timeout = 5)
    delcount = delcount + 1
print (delcount "certs deleted")

In [ ]:
#Display any certificates after deletion
df_cert_exp

### 4. Restart the tomcat service

In [165]:
#Restart tomcat services
output.clear()
interact.send(" ")
interact.expect('admin:')
interact.send("utils service restart Cisco Tomcat")
interact.send(" ")
interact.expect('admin:')

 
admin:utils service restart Cisco Tomcat
Do not press Ctrl+C while the service is restarting. If the service has not restarted properly, execute the same command again.
Service Manager is running
Cisco Tomcat[STOPPING]
Cisco Tomcat[STOPPING]
Cisco Tomcat[STOPPING]
Cisco Tomcat[STOPPING]
Cisco Tomcat[STARTING]
Cisco Tomcat[STARTING]
Cisco Tomcat[STARTED]
admin:

0